In [35]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt


# 1. 데이터 로드
region_df = pd.read_csv(r"C:\Users\krtc0\OneDrive\문서\GitHub\sky-root-analysis\result\region\2024-08-19.csv")
emd = gpd.read_file(r"C:\Users\krtc0\OneDrive\문서\GitHub\sky-root-analysis\data\open\emd_WGS84.json")

# 2. 코드 타입 통일
region_df["EMD_CODE"] = region_df["EMD_CODE"].astype(int).astype(str)
emd["EMD_CD"] = emd["EMD_CD"].astype(str)

# 3. 조인
gdf = emd.merge(region_df, left_on="EMD_CD", right_on="EMD_CODE", how="left")

# # 4. 시각화
# fig, ax = plt.subplots(1, 1, figsize=(10, 10))
# gdf.plot(
#     column="severity_score",
#     cmap="Reds",
#     legend=True,
#     ax=ax,
#     missing_kwds={"color": "lightgrey"}
# )

# ax.set_title("지역별 개선 필요도 (Severity Score)")
# ax.axis("off")
# plt.show()

In [36]:
exclude_emds = {"28110147", "11500108"} # 인천공항, 김포공항

top3_codes = (
    region_df[region_df["improve_trips"] == True]
    .sort_values("total_trips", ascending=False)
    .loc[lambda df: ~df["EMD_CODE"].astype(str).isin(exclude_emds)]
    .head(1)["EMD_CODE"]
    .astype(str)
    .tolist()
)

In [37]:
# 🗺️ Folium 기반 시각화 (추천 ⭐)

import folium
from shapely.geometry import LineString

m = folium.Map(location=[37.55, 126.98], zoom_start=12)

for top3_code in top3_codes:
# 기준 지역
    emd_target = gdf[gdf["EMD_CODE"] == top3_code]

    folium.GeoJson(
        emd_target.geometry,
        name=f"EMD {top3_code}",
        style_function=lambda x: {
            "fillColor": "none",
            "color": "red",
            "weight": 3,
            "tooltip": f"{top3_code}"
        }
    ).add_to(m)
    
folium.LayerControl().add_to(m)

In [38]:
# 🧍 실제 이동 경로 (파란 선)
trip_df = pd.read_csv(r"C:\Users\krtc0\OneDrive\문서\GitHub\sky-root-analysis\data\processed\2024-08-19.csv")

for top3_code in top3_codes:

    for trip_no, group in trip_df.groupby("TRIP_NO"):
        start_emd = group.iloc[0]["EMD_CODE"].astype(str)
        
        if start_emd != top3_code:
            continue   # 시작 EMD가 아니면 skip
        
        coords = list(zip(group["DPR_CELL_YCRD"], group["DPR_CELL_XCRD"]))
        
        folium.PolyLine(
            coords,
            color="black",
            weight=2,
            opacity=0.6,
            tooltip=f"{trip_no}"
            
        ).add_to(m)

C:\Users\krtc0\AppData\Local\Temp\ipykernel_8064\834534854.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  trip_df = pd.read_csv(r"C:\Users\krtc0\OneDrive\문서\GitHub\sky-root-analysis\data\processed\2024-08-19.csv")


In [39]:
for top3_code in top3_codes:

    for trip_no, trip_group in trip_df.groupby("TRIP_NO"):

        # 1️⃣ 시작 EMD 필터
        start_emd = trip_group.iloc[0]["EMD_CODE"].astype(str)
        if start_emd != top3_code:
            continue   # 시작 EMD가 아니면 skip

        # 2️⃣ 이 TRIP 안에서 버스 이벤트만 추출
        bus_events = trip_group[trip_group["BUS_ID"].notna()]

        if bus_events.empty:
            continue

        # 3️⃣ 같은 BUS_ID 단위로 묶기
        for bus_id, bus_group in bus_events.groupby("BUS_ID"):

            start_time = bus_group["DPR_MT1_UNIT_TM"].min()
            end_time   = bus_group["ARV_MT1_UNIT_TM"].max()

            # 4️⃣ 버스 탑승 구간 전체 경로
            full_path = trip_group[
                (trip_group["DPR_MT1_UNIT_TM"] >= start_time) &
                (trip_group["DPR_MT1_UNIT_TM"] <= end_time)
            ].sort_values("DPR_MT1_UNIT_TM")

            if len(full_path) < 2:
                continue

            coords = list(zip(
                full_path["DPR_CELL_YCRD"],
                full_path["DPR_CELL_XCRD"]
            ))

            bus_type = bus_group["TRANSPORT_TYPE"].iloc[0]

            color_map = {
                "일반버스": "red",
                "공항버스": "blue"
            }

            folium.PolyLine(
                coords,
                color=color_map.get(bus_type, "black"),
                weight=2,
                opacity=0.6,
                tooltip=f"{trip_no} | {bus_type} | {bus_id}"
            ).add_to(m)


In [40]:
m.save("region_route.html")